# Learning private models with multiple teachers

Protocol:
1. Train teachers:
    - Devide training set into buckets (not overlapping)
    - Train a models (teacher) on each bucket
2. Train student:
    - Extract a share of the test set
    - Ensemble predictions from teachers: queries each teacher for predictions on the test set share
    - Aggregate teacher predictions to get student training labels using noising max: it
  adds Laplacian noise to label counts and returns the most frequent label
    - Train student with the aggregated label
    - Validate the student model on the remaining test data

http://www.cleverhans.io/privacy/2018/04/29/privacy-and-machine-learning.html
https://github.com/tensorflow/models/tree/master/research/differential_privacy/multiple_teachers

In [1]:
from pate import train_teachers, train_student, prepare_mnist

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
dataset = "mnist" 
nb_labels = 10
nb_teachers = 100 
stdnt_share = 1000
lap_scale = 10

In [3]:
# For this demo, we use MNIST dataset
train_data, train_labels, test_data, test_labels = prepare_mnist()

In [4]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5, stride = 1)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.avgpool1 = nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(16, 16, 5, stride = 1)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.relu2 = nn.ReLU()
        self.avgpool2 = nn.AvgPool2d(2)
        self.linear1 = nn.Linear(256, 100)
        self.batchnorm3 = nn.BatchNorm1d(100)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.relu1(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.relu2(x)
        x = self.avgpool2(x)
        x = x.view(-1, 256)
        x = self.linear1(x)
        x = self.batchnorm3(x)
        x = self.relu3(x)
        out = self.linear2(x)
        return out

## Train Teachers

In [6]:
ckpt_path = 'checkpoint/'

for teacher_id in range(nb_teachers):
    
    filename = str(dataset) + '_' + str(nb_teachers) + '_teachers_' + str(teacher_id) + '.pth'
    
    model = CNN_Model(nb_labels)
    
    train_teachers(model, train_data, train_labels, test_data, test_labels, nb_teachers, teacher_id, filename)


Train teacher ID: 0


/anaconda3/envs/py27/lib/python2.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Accuracy: 368/600 (61%)
Train Accuracy: 533/600 (89%)
Train Accuracy: 565/600 (94%)
Train Accuracy: 588/600 (98%)
Train Accuracy: 595/600 (99%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9625/10000 (96%)

Train teacher ID: 1
Train Accuracy: 363/600 (60%)
Train Accuracy: 540/600 (90%)
Train Accuracy: 576/600 (96%)
Train Accuracy: 591/600 (98%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9512/10000 (95%)

Train teacher ID: 2
Train Accuracy: 420/600 (70%)
Train Accuracy: 566/600 (94%)
Train Accuracy: 583/600 (97%)
Train Accuracy: 594/600 (99%)
Train Accuracy: 594/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accu

Train Accuracy: 367/600 (61%)
Train Accuracy: 543/600 (90%)
Train Accuracy: 565/600 (94%)
Train Accuracy: 589/600 (98%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9458/10000 (95%)

Train teacher ID: 24
Train Accuracy: 378/600 (63%)
Train Accuracy: 542/600 (90%)
Train Accuracy: 586/600 (98%)
Train Accuracy: 594/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9543/10000 (95%)

Train teacher ID: 25
Train Accuracy: 381/600 (64%)
Train Accuracy: 542/600 (90%)
Train Accuracy: 570/600 (95%)
Train Accuracy: 584/600 (97%)
Train Accuracy: 594/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 599/600 (100%)
Train 

Test Accuracy: 9554/10000 (96%)

Train teacher ID: 46
Train Accuracy: 376/600 (63%)
Train Accuracy: 553/600 (92%)
Train Accuracy: 576/600 (96%)
Train Accuracy: 591/600 (98%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Test Accuracy: 9429/10000 (94%)

Train teacher ID: 47
Train Accuracy: 378/600 (63%)
Train Accuracy: 549/600 (92%)
Train Accuracy: 576/600 (96%)
Train Accuracy: 589/600 (98%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9601/10000 (96%)

Train teacher ID: 48
Train Accuracy: 392/600 (65%)
Train Accuracy: 540/600 (90%)
Train Accuracy: 571/600 (95%)
Train Accuracy: 578/600 (96%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 6

Train Accuracy: 597/600 (100%)
Test Accuracy: 9509/10000 (95%)

Train teacher ID: 69
Train Accuracy: 345/600 (58%)
Train Accuracy: 546/600 (91%)
Train Accuracy: 574/600 (96%)
Train Accuracy: 591/600 (98%)
Train Accuracy: 592/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Test Accuracy: 9480/10000 (95%)

Train teacher ID: 70
Train Accuracy: 340/600 (57%)
Train Accuracy: 533/600 (89%)
Train Accuracy: 569/600 (95%)
Train Accuracy: 589/600 (98%)
Train Accuracy: 595/600 (99%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9471/10000 (95%)

Train teacher ID: 71
Train Accuracy: 384/600 (64%)
Train Accuracy: 552/600 (92%)
Train Accuracy: 574/600 (96%)
Train Accuracy: 590/600 (98%)
Train Accuracy: 594/600 (99%)
Train Accuracy: 593/600 (99%)
Train Accuracy: 6

Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9486/10000 (95%)

Train teacher ID: 92
Train Accuracy: 384/600 (64%)
Train Accuracy: 537/600 (90%)
Train Accuracy: 577/600 (96%)
Train Accuracy: 591/600 (98%)
Train Accuracy: 593/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9509/10000 (95%)

Train teacher ID: 93
Train Accuracy: 394/600 (66%)
Train Accuracy: 548/600 (91%)
Train Accuracy: 574/600 (96%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 592/600 (99%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 9533/10000 (95%)

Train teacher ID: 94
Train Accuracy: 398/600 (66%)
Train Accuracy: 548/600 (91%)
Train Accuracy: 572/600 (95%)
Train Accuracy: 590/600 (98%)
Train Accuracy: 593/600 (99%)
Train Accuracy: 

## Train Student

In [7]:
model = CNN_Model(nb_labels)

train_student(model, dataset, nb_labels, nb_teachers, stdnt_share, lap_scale)

pate.py:191: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_softmax = F.softmax(output).data.numpy()


Computed Teacher 0 softmax predictions
Computed Teacher 1 softmax predictions
Computed Teacher 2 softmax predictions
Computed Teacher 3 softmax predictions
Computed Teacher 4 softmax predictions
Computed Teacher 5 softmax predictions
Computed Teacher 6 softmax predictions
Computed Teacher 7 softmax predictions
Computed Teacher 8 softmax predictions
Computed Teacher 9 softmax predictions
Computed Teacher 10 softmax predictions
Computed Teacher 11 softmax predictions
Computed Teacher 12 softmax predictions
Computed Teacher 13 softmax predictions
Computed Teacher 14 softmax predictions
Computed Teacher 15 softmax predictions
Computed Teacher 16 softmax predictions
Computed Teacher 17 softmax predictions
Computed Teacher 18 softmax predictions
Computed Teacher 19 softmax predictions
Computed Teacher 20 softmax predictions
Computed Teacher 21 softmax predictions
Computed Teacher 22 softmax predictions
Computed Teacher 23 softmax predictions
Computed Teacher 24 softmax predictions
Computed T

True